In [37]:
from PIL import Image, ImageOps, ImageFilter
import pandas as pd 
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt

## Data Seperation part

In [6]:
TRAIN=r'D:\major_project\megha\project-1\processed\train'
TEST=r'D:\major_project\megha\project-1\processed\test'

In [8]:
import os
import pandas as pd
files_path=pd.DataFrame(columns=['Image','label'])
def get_files_from_path(directory):
    
    for i in os.listdir(directory):
    
        for j in os.listdir(f"{directory}\\{i}"):
            

            path=os.path.join('processed','train',i,j)

            number=len(files_path)
            files_path.at[number,'Image']=path
            files_path.at[number,'label']=int(i)





        
        
    
if os.path.exists(r'D:\major_project\megha\project-1\processed\train\train_images.csv'):
    train=pd.read_csv(r'D:\major_project\megha\project-1\processed\train\train_images.csv')
else:
    
    get_files_from_path(TRAIN)
    files_path.to_csv(r'D:\major_project\megha\project-1\processed\train\train_images.csv',index=False)


In [9]:
import os
import pandas as pd
files_path=pd.DataFrame(columns=['Image','label'])
def get_files_from_path(directory):
    
    for i in os.listdir(directory):
    
        for j in os.listdir(f"{directory}\\{i}"):
            

            path=os.path.join('processed','test',i,j)

            number=len(files_path)
            files_path.at[number,'Image']=path
            files_path.at[number,'label']=int(i)




        
        
    
if os.path.exists(r'D:\major_project\megha\project-1\processed\test\test_images.csv'):
    test=pd.read_csv(r'D:\major_project\megha\project-1\processed\test\test_images.csv')
else:
    
    get_files_from_path(TRAIN)
    files_path.to_csv(r'D:\major_project\megha\project-1\processed\test\test_images.csv',index=False)

In [10]:
train.head()

,Image,label
0,processed\train\0\000u100t01.tiff,0
1,processed\train\0\000u100t02.tiff,0
2,processed\train\0\000u103t01.tiff,0
3,processed\train\0\000u103t02.tiff,0
4,processed\train\0\000u104t01.tiff,0


## Image loading part and apply preprocesising

In [48]:
import cv2
def preprocess(image_path):

    numpy_img=cv2.imread(image_path)
    # Convert NumPy array to PIL image
    img = Image.fromarray(numpy_img)

    # Convert to grayscale
    converted = img.convert("L")

    # Invert colors
    inverted = ImageOps.invert(converted)

    # Apply maximum filter
    thick = inverted.filter(ImageFilter.MaxFilter(5))

    # Calculate resizing ratio (adjusting for smaller size)
    ratio = 32.0 / max(thick.size)

    # Resize image
    new_size = tuple([int(round(x*ratio)) for x in thick.size])
    res = thick.resize(new_size, Image.LANCZOS)
    
    # Convert resized image back to NumPy array
    arr = np.asarray(res)

    # Calculate center of mass
    com = ndimage.center_of_mass(arr)

    # Create blank image
    result = np.zeros((64, 64), dtype=np.uint8)

    # Calculate paste box coordinates
    box = (int(round(32.0 - com[1])), int(round(32.0 - com[0])))

    # Paste resized image onto blank image
    result[box[1]:box[1]+arr.shape[0], box[0]:box[0]+arr.shape[1]] = arr

    return result

In [49]:
train.shape

(50683, 2)

In [50]:
train_shuffled=train.sample(train.shape[0]).reset_index().iloc[:,1:].copy()

In [51]:
X=[]
y=[]


for index,row in train_shuffled.iterrows():
    image_path=row[0]
    label=row[1]

    pre_img=preprocess(image_path)
    

    X.append(pre_img)
    y.append(label)





In [60]:
X=np.array(X).reshape(-1,64,64,1)

In [61]:
np.unique(y)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155])

In [62]:
import pickle 
pickle_out=open(r'D:\major_project\megha\project-1\data_final\train\X.pickle','wb')
pickle.dump(X,pickle_out)
pickle_out.close()


pickle_out=open(r'D:\major_project\megha\project-1\data_final\train\y.pickle','wb')
pickle.dump(y,pickle_out)
pickle_out.close()


In [63]:
test.shape

(26926, 2)

In [66]:
test_shuffled=test.sample(test.shape[0]).reset_index().iloc[:,1:].copy()

In [71]:
X_test=[]
y_test=[]

print_label=None
for index,row in test_shuffled.iterrows():
    
    image_path=row[0]
    label=row[1]

    pre_img=preprocess(image_path)
    

    X_test.append(pre_img)
    y_test.append(label)


    

    


In [73]:
X_test=np.array(X_test).reshape(-1,64,64,1)

In [74]:
np.unique(y_test)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155])

In [75]:
pickle_out=open(r'D:\major_project\megha\project-1\data_final\test\X_test.pickle','wb')
pickle.dump(X_test,pickle_out)
pickle_out.close()


pickle_out=open(r'D:\major_project\megha\project-1\data_final\test\y_test.pickle','wb')
pickle.dump(y_test,pickle_out)
pickle_out.close()